## Imports

In [69]:
import pandas as pd
import numpy as np
import configparser
import os
import sys

In [70]:
cwd = os.getcwd()

In [71]:
module_path = os.path.join(cwd, '..')
if module_path not in sys.path:
    sys.path.append(module_path)

In [72]:
import financialstatements

## Config

In [73]:
#Creating configuration
config_path = os.path.join(cwd, '../docs/config.ini')
config = configparser.ConfigParser()
config.read(config_path)

['c:\\Users\\tasco\\OneDrive\\Python\\Projects\\financialstatements\\financialstatements\\notebooks\\../docs/config.ini']

## Reading Data

In [74]:
GENERAL_LEDGER = os.path.join(cwd, config.get("data_outputs_directory", "GENERAL_LEDGER"))
MONTH_DATA = os.path.join(cwd, config.get('table_files', 'MONTH_DATA'))

In [75]:
paths = financialstatements.creating_input_paths(GENERAL_LEDGER)

In [ ]:
dfs = []
for path in paths:
    df = pd.read_csv(path)
    dfs.append(df)

In [76]:
month_df = pd.read_excel(MONTH_DATA)

## ETL

1. General Ledger per month
2. Trial balance per month
3. Closing entries per month